cd luca/Scrivania/Magistrale/SistemiDigitali/Progetto/Project_SistemiDigitali/AES_Encryption/Vivado/AES_CTR_Pynq/
scp design_1.bit xilinx@192.168.2.99:/home/xilinx/pynq/overlays/AES_Encryption/"
scp design_1.tcl xilinx@192.168.2.99:/home/xilinx/pynq/overlays/AES_Encryption/"
scp design_1.hwh xilinx@192.168.2.99:/home/xilinx/pynq/overlays/AES_Encryption/"

scp 

In [ ]:
from pynq import Overlay
import pynq.lib.dma
import numpy as np
import time

# Caricamento dell'overlay
overlay = Overlay('/home/xilinx/pynq/overlays/AES_Encryption/design_1.bit')

# Caricamento del DMA
dma = overlay.axi_dma_0

# Check DMA status
sendstatus = dma.sendchannel.running
recvstatus = dma.recvchannel.running
print("sendstatus:", sendstatus," -  recvstatus:",recvstatus)

# Alloca memoria per l'input e l'output -> 2 array da 16 byte (uint8)
data_size = 16
in_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)
out_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)


# Plaintext di prova
#plaintext = b"\x04\xb4\x56\xee\x17\xa6\x6a\xbe\x53\x28\xa7\x73\xd6\x7c\xd8\x6a"
plaintext = b"\x33\xdd\x56\xee\x17\xa6\x1a\x13\x32\x28\xa7\x73\x65\x1c\xd2\x41"
plaintext_array = np.array(bytearray(plaintext))
print("Plaintext: ",plaintext)

# Copia dei dati nel buffer
np.copyto(in_buffer, plaintext_array)

# Avviare il trasferimento del DMA ed aspettare i risultati
start = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
end = time.time()

print("Execution time:", end - start, "s")

output = bytearray(out_buffer)
print("Cipher:", bytes(output))

# Dovrebbe venire
#       a9 99 07 o    e3 f3 bf X    a4 b2 b0 [    e5 | g a4
#       a9 99 07 6f   e3 f3 bf 58   a4 b2 b0 5b   e5 7c 67 a4 

# Liberare i buffer
in_buffer.close()
out_buffer.close()



Per nascondere all'utente il codice di gestione del DMA, possiamo scrivere un driver 

In [ ]:
from pynq import Default